In [298]:
import pandas as pd


In [299]:
glassdoor_file = 'resources/glassdoor.csv'
df = pd.read_csv(glassdoor_file)
drop_col = df.columns
drop_col

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'hourly', 'employer_provided', 'min_salary', 'max_salary', 'avg_salary',
       'company_txt', 'job_state', 'same_state', 'age', 'python_yn', 'R_yn',
       'spark', 'aws', 'excel'],
      dtype='object')

In [300]:
b = ['Job Title', 'Job Description', 'Rating', 'Location', 'Headquarters', 'Size', 'Founded', 'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors', 'hourly', 'employer_provided', 'min_salary', 'max_salary', 'avg_salary', 'company_txt', 'job_state', 'same_state', 'age', 'python_yn', 'R_yn', 'spark', 'aws', 'excel']
df.drop(axis=1, columns=b, inplace=True)
df.head()

,Salary Estimate,Company Name
0,$53K-$91K (Glassdoor est.),Tecolote Research\n3.8
1,$63K-$112K (Glassdoor est.),University of Maryland Medical System\n3.4
2,$80K-$90K (Glassdoor est.),KnowBe4\n4.8
3,$56K-$97K (Glassdoor est.),PNNL\n3.8
4,$86K-$143K (Glassdoor est.),Affinity Solutions\n2.9


In [301]:
df['Company Name'].unique

<bound method Series.unique of 0                                 Tecolote Research\n3.8
1             University of Maryland Medical System\n3.4
2                                           KnowBe4\n4.8
3                                              PNNL\n3.8
4                                Affinity Solutions\n2.9
5                                          CyrusOne\n3.4
6                                ClearOne Advantage\n4.1
7                                        Logic20/20\n3.8
8                         Rochester Regional Health\n3.3
9                                          <intent>\n4.6
10                                             Wish\n3.5
11                                          ManTech\n4.1
12                                          Walmart\n3.2
13                                           Yesler\n4.1
14                           Takeda Pharmaceuticals\n3.7
15                                          Audible\n3.6
16                              Blueprint Medicines\n3.9


In [302]:
df.drop_duplicates(subset=['Company Name'],inplace=True)
df.reset_index(inplace=True, drop=True)
df

,Salary Estimate,Company Name
0,$53K-$91K (Glassdoor est.),Tecolote Research\n3.8
1,$63K-$112K (Glassdoor est.),University of Maryland Medical System\n3.4
2,$80K-$90K (Glassdoor est.),KnowBe4\n4.8
3,$56K-$97K (Glassdoor est.),PNNL\n3.8
4,$86K-$143K (Glassdoor est.),Affinity Solutions\n2.9
5,$71K-$119K (Glassdoor est.),CyrusOne\n3.4
6,$54K-$93K (Glassdoor est.),ClearOne Advantage\n4.1
7,$86K-$142K (Glassdoor est.),Logic20/20\n3.8
8,$38K-$84K (Glassdoor est.),Rochester Regional Health\n3.3
9,$120K-$160K (Glassdoor est.),<intent>\n4.6


In [303]:
df1 = df['Company Name'].str.split(pat='\n',expand=True)

In [304]:
df = pd.concat([df, df1],axis=1)
df

,Salary Estimate,Company Name,0,1
0,$53K-$91K (Glassdoor est.),Tecolote Research\n3.8,Tecolote Research,3.8
1,$63K-$112K (Glassdoor est.),University of Maryland Medical System\n3.4,University of Maryland Medical System,3.4
2,$80K-$90K (Glassdoor est.),KnowBe4\n4.8,KnowBe4,4.8
3,$56K-$97K (Glassdoor est.),PNNL\n3.8,PNNL,3.8
4,$86K-$143K (Glassdoor est.),Affinity Solutions\n2.9,Affinity Solutions,2.9
5,$71K-$119K (Glassdoor est.),CyrusOne\n3.4,CyrusOne,3.4
6,$54K-$93K (Glassdoor est.),ClearOne Advantage\n4.1,ClearOne Advantage,4.1
7,$86K-$142K (Glassdoor est.),Logic20/20\n3.8,Logic20/20,3.8
8,$38K-$84K (Glassdoor est.),Rochester Regional Health\n3.3,Rochester Regional Health,3.3
9,$120K-$160K (Glassdoor est.),<intent>\n4.6,<intent>,4.6


In [305]:
df.drop(axis=1, columns=['Company Name', 1],inplace=True)

In [306]:
df.rename(columns={0:"Company Name"},inplace=True)
df

,Salary Estimate,Company Name
0,$53K-$91K (Glassdoor est.),Tecolote Research
1,$63K-$112K (Glassdoor est.),University of Maryland Medical System
2,$80K-$90K (Glassdoor est.),KnowBe4
3,$56K-$97K (Glassdoor est.),PNNL
4,$86K-$143K (Glassdoor est.),Affinity Solutions
5,$71K-$119K (Glassdoor est.),CyrusOne
6,$54K-$93K (Glassdoor est.),ClearOne Advantage
7,$86K-$142K (Glassdoor est.),Logic20/20
8,$38K-$84K (Glassdoor est.),Rochester Regional Health
9,$120K-$160K (Glassdoor est.),<intent>


In [307]:
df2 = df
df1 = df2['Salary Estimate'].str.rstrip('(Glassdoor est.)')
df.drop(axis=1, columns=['Salary Estimate'],inplace=True)
df = pd.concat([df, df1],axis=1)

In [308]:
df.head()

,Company Name,Salary Estimate
0,Tecolote Research,$53K-$91K
1,University of Maryland Medical System,$63K-$112K
2,KnowBe4,$80K-$90K
3,PNNL,$56K-$97K
4,Affinity Solutions,$86K-$143K


In [309]:
mask = df['Company Name'].str.contains('LLC').value_counts()

In [310]:
df.drop(mask, inplace=True)

In [311]:
pd.set_option('display.max_rows',None,'display.max_columns',None)

In [312]:
To_remove_lst = [' .LLC', ' INC.', ' LLC', ' ,INC', ' INC', ' Inc.', ', Inc.', ', Inc']

df['Company Name'] = df['Company Name'].str.replace('|'.join(To_remove_lst), '')

To_remove_lst = [' Inc.', ', Inc.', ', Inc', '. Inc']

df['Company Name'] = df['Company Name'].str.replace('|'.join(To_remove_lst), ',')
df['Company Name'] = df['Company Name'].str.upper()
df['Salary Estimate'] = df['Salary Estimate'].str.replace('|'.join(['Employer Provided Salary:', '( ', ')', 'Employ', 'PerHou'] ), '')
A = df['Salary Estimate'].str.split('-',expand=True)
A[0] = A[0].str.replace('(\D+)','')
A[1] = A[1].str.replace('(\D+)','')


A.rename(columns={0: "Lower Salary Range (K)", 1: "Upper Salary Range (K)"},inplace=True)
df = pd.concat([df, A],axis=1)
df.drop(axis=1, columns=['Salary Estimate'], inplace=True)

In [313]:
df

,Company Name,Lower Salary Range (K),Upper Salary Range (K)
0,TECOLOTE RESEARCH,53,91
1,UNIVERSITY OF MARYLAND MEDICAL SYSTEM,63,112
2,KNOWBE4,80,90
3,PNNL,56,97
4,AFFINITY SOLUTIONS,86,143
5,CYRUSONE,71,119
6,CLEARONE ADVANTAGE,54,93
8,ROCHESTER REGIONAL HEALTH,38,84
9,<INTENT>,120,160
10,WISH,126,201


In [314]:
df.to_csv('cleaned_glassdoor.csv', index=False)